In [ ]:
#Codigo para clasificar mensajes recibidos + matchear
#Ultima actualizacion: 04/Ago/2022

# Inputs: 
#       (A) sent_mails_conf_envio_[FECHA] (del codigo anterior), que tiene las respuestas que se enviaron efectivamente y 
#       (B) base de mails con respuestas de inmobiliarias (puede haber de properati, no es problema).
# Outputs: 
#       (A) respuestas_inmobiliarias_[FECHA], que agrega TODAS las publicaciones y respuestas, matcheadas o no (despues se puede filtrar por lo que queramos), y 
#       (B) resultados_matching_rtas_[FECHA] que muestra, para cada mail, la cantidad de respuestas matcheadas y los % de match.


In [5]:
# Importar paquetes necesarios para la clasificacion

import pandas as pd
import numpy as np
from datetime import datetime
import matching_v1 as matching
import tools

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

#Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)

# Ronda y fecha de la ronda
ronda = '2'
fecha = '2022_05_03'

#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#### Rutas utilizadas:
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_dataout = path_data + r"\mails_matches\matched\respuestas_inmobiliarias\round2"
path_datain = path_data + r"\mails_matches"
path_deleted = path_data + r"\mails_matches"

In [6]:
#Importar base de mails recibidos (todos)
rtas_full = pd.read_excel(path_datain + fr"\to_match\mails_total_2022_05_19.xlsx") # 2022_05_05 Ultima descarga de rtas de gmail

#Importar base de mails con confirmación de envios
enviados = pd.read_excel(path_datain + fr"\matched\confirmaciones_de_envio\sent_mails_conf_envio_round_{ronda}_2022_08_04.xlsx")
enviados = enviados.drop(columns=['Unnamed: 0.1','_merge'])
enviados = enviados[-enviados.body.str.contains("miberniel")] # Elimino las observaciones que la propiedad de Inés

In [7]:
#### JULI: FIXME: Buscar en qué base de randomization existe una propiedad de Solimano Dal Pos que se haya enviado desde fh.pablogomez@gmail.com.
display(rtas_full[rtas_full['thread_id'] == '180899855a25094a'])


#### JULI: FIXME: Buscar en qué base de randomization existe una propiedad de Solimano Dal Pos que se haya enviado desde fh.pablogomez@gmail.com.
# display(rtas_full[rtas_full['thread_id'] == '180899855a25094a'])

,Unnamed: 0.1,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,pais,index
1969,1969,1969,Casas A ESTRENAR en Barrio Privado,Solimano Dal Pos <info@solimanodalpos.com.ar>,fh.pablogomez@gmail.com,[<p>Visita la versiÃ³n online para ver el cont...,"Tue, 3 May 2022 11:05:02 +0000 (UTC)",NaN,180899855a25094a,multipart/alternative,arg,NaN


In [8]:
#### Elimino observaciones que se corresponden a mails de prueba o de Google

# Dropeo observaciones que tienen Google en el mail / user
prev = len(rtas_full)
rtas_full_out = rtas_full[rtas_full['from'].str.contains("Google", na=False) == True]
rtas_full =     rtas_full[rtas_full['from'].str.contains("Google", na=False) == False]
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba de Juli
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]
rtas_full = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]])

print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == True]
rtas_full = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Base de correos desde los que enviamos
mails_df = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\0_instrucciones\2-Emails para Intervención.xlsx")
mails_df['Usuario_clean'] = mails_df.Usuario.apply(matching.clean_mail)
users = mails_df['Usuario_clean'].to_list()

# Dropeo observaciones de prueba (from == to, ¡nos los autoenviamos!)
prev = len(rtas_full) 
for user in users:
    to_concat = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==True ]
    rtas_full = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==False]
    rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

Se dropearon 423 observaciones.
Se dropearon 221 observaciones.
Se dropearon 146 observaciones.
Se dropearon 656 observaciones.


In [9]:
##### Identifica mails que vienen de properati 'contacts@mailing.properati.com'

# Filtro nulls
before = len(rtas_full)
rtas_full = rtas_full.dropna(subset=['from','body'])
after = len(rtas_full)
print("Se dropearon", before-after, "observaciones.")

# Creo binaria con True para los mails de properati
mails_properati = ['contacts@mailing.properati.com']

rtas_full['conf_envio'] = rtas_full['from'].apply(matching.match_string_con_lista, lista_matchs=mails_properati)
print("Match confirmaciones realizado.\nResultados:")

# Reporta los resultados
rtas_full['conf_envio'].value_counts()

Se dropearon 20 observaciones.
Match confirmaciones realizado.
Resultados:


True     7281
False    4931
Name: conf_envio, dtype: int64

In [10]:
enviados = enviados[enviados.mail.notnull()]

In [11]:
##### Limpio las fechas
rtas_full['date_clean'] = rtas_full['date'].str.replace(".+,\s","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("\-|\+|(0\d00)(.*)","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("GMT","")
rtas_full['date_clean'] = rtas_full['date_clean'].str.lstrip(' ').str.rstrip(' ')
rtas_full['date_clean'] = rtas_full['date_clean'].str.zfill(20)
rtas_full['date_clean'] = pd.to_datetime(rtas_full['date_clean'], format="%d %b %Y %H:%M:%S")
rtas_full['date_clean'] = rtas_full['date_clean'].dt.date  # Solo me quedo dia-mes-año

In [12]:
##### Matcheo respuestas (ya mergeadas con @properati) con scraps
from importlib import reload
reload(matching)
# lista_paises=[]
fin_ronda_1 = pd.to_datetime("2022-05-03").date()
rtas_inmob = rtas_full.loc[(rtas_full['date_clean']>=fin_ronda_1) & (rtas_full['conf_envio']==False)]

today = datetime.now()
today = today.strftime("%Y_%m_%d")
nombre = f'mails_total_matched_round_{ronda}_{today}'

rtas_inmob['index_scrap'] = np.nan
rtas_inmob['match_confiab'] = np.nan
rtas_inmob['match_step'] = np.nan
rtas_inmob['match_string'] = np.nan
rtas_inmob['match2_string'] = np.nan
enviados['index_rtas'] = np.nan
enviados['match_confiab'] = np.nan
enviados['match_step'] = np.nan


#### FIXME: ¿Por qué desaparecen observaciones?
mails = matching.full_proceso(path=path_dataout, rtas_full=rtas_inmob, scrap_full=enviados)


###########################################################
Cantidad de Respuestas: 2529 
###########################################################
Limpio nombre de inmobiliarias, precios y calles:
palabras eliminadas: ['en', 'alquiler', 'departamento', '', 'al', 'y', 'de', 'entre', 'apartamento', 'arriendo', 'av.', 'san']
palabras eliminadas: []
palabras eliminadas: ['propiedades', 'inmobiliaria', '', 'inmobiliarios', 'negocios', 'bienes', 'sas', 'raices', 'inmobiliario']
Despues de la limpieza... ¿hay repetidas?
False    2666
True      970
Name: inmobiliaria_c, dtype: int64


100%|██████████| 192/192 [08:14<00:00,  2.57s/it] 


C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\mails_matches\matched\respuestas_inmobiliarias\round2\resultados_matching_rtas_2022_08_11.xlsx creado
C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\mails_matches\matched\respuestas_inmobiliarias\round2\respuestas_inmobiliarias_2022_08_11_matched.xlsx creado
end of script


In [ ]:
mails_rec = pd.unique(list(enviados.mail.apply(matching.clean_mail))).tolist()

print([h for h in mails if h not in mails_rec])	
print([h for h in mails_rec if h not in mails])

In [18]:
mails_rec = pd.unique(list(enviados.mail.apply(matching.clean_mail))).tolist()
mails_rec

['pablom.ngomez@gmail.com',
 'jcmanuelrodrigue.z@gmail.com',
 'gome.zpablou@gmail.com',
 'rodriguez.j.cmanuel@gmail.com',
 'pablom.ngome.z@gmail.com',
 'rodriguez.manuel.j.f@gmail.com',
 'ro.driguezmanuelw@gmail.com',
 'manuelrodrigue.zjf@gmail.com',
 'rodriguez.j.c.manuel@gmail.com',
 'gomezf.hpablo@gmail.com',
 'f.h.pablogomez@gmail.com',
 'rodriguez.manuelj.f@gmail.com',
 'p.hpablo.gomez@gmail.com',
 'pablogome.z.f.h@gmail.com',
 'rodriguezj.cmanuel@gmail.com',
 'pablo.gomez.f.h@gmail.com',
 'jfmanuelrodrigue.z@gmail.com',
 'pablogome.zf.h@gmail.com',
 'j.fmanuelrodriguez@gmail.com',
 'p.hpablogome.z@gmail.com',
 'j.fmanuel.rodriguez@gmail.com',
 'f.h.pablo.gomez@gmail.com',
 'rodrigue.zmanuel.w@gmail.com',
 'p.h.pablo.gomez@gmail.com',
 'rodrigue.z.manuel.w@gmail.com',
 'rodrigue.zmanuelw@gmail.com',
 'rodriguezmanuelj.f@gmail.com',
 'gomezf.h.pablo@gmail.com',
 'f.hpablogomez@gmail.com',
 'j.cmanuelrodriguez@gmail.com',
 'manuel.rodriguez.j.f@gmail.com',
 'p.h.pablogomez@gmail.com

In [16]:
# from_val = 'Solimano Dal Pos <info@solimanodalpos.com.ar>'
# thread_val = '180899855a25094a'
# display(rtas_inmob[(rtas_inmob['from']==from_val) & (rtas_inmob['thread_id']==thread_val)])
# from_val = 'Eliana Avila <eliana@avilas.com.ar>'
thread_val = '1809469d38faec3c'
display(rtas_inmob[(rtas_inmob['thread_id']==thread_val)])

,Unnamed: 0.1,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,pais,index,conf_envio,date_clean,index_scrap,match_confiab,match_step,match_string,match2_string
7547,3180,3180,Respuesta a tu consulta - Propiedades de Alqui...,Propiedades de Alquiler y Venta - Contacto <bb...,f.garcia.juan.p@gmail.com,[<p>@import url(http://url2214.inmomail.com/ls...,"Thu, 05 May 2022 13:30:03 +0000 (UTC)",NaN,1809469d38faec3c,multipart/alternative,ecu,24.0,False,2022-05-05,NaN,NaN,NaN,NaN,NaN
7548,3181,3181,Respuesta a tu consulta - Propiedades de Alqui...,Propiedades de Alquiler y Venta - Contacto <bb...,f.garcia.juan.p@gmail.com,[<p>@import url(http://url2214.inmomail.com/ls...,"Thu, 05 May 2022 13:30:03 +0000 (UTC)",NaN,1809469d38faec3c,multipart/alternative,ecu,25.0,False,2022-05-05,NaN,NaN,NaN,NaN,NaN


In [ ]:
rtas_inmob[(rtas_inmob['thread_id']==thread_val)].body.values

In [ ]:
### FIXME: Notar que Solimano está en mails diferentes... AYUDA
enviados[(enviados.descripcion.str.contains("olimano"))]

In [19]:
##### FIXME: ¿Por qué mails es distinto de mails2???
mails = pd.unique(list(enviados.mail.apply(matching.clean_mail))).tolist()
mails

['pablom.ngomez@gmail.com',
 'jcmanuelrodrigue.z@gmail.com',
 'gome.zpablou@gmail.com',
 'rodriguez.j.cmanuel@gmail.com',
 'pablom.ngome.z@gmail.com',
 'rodriguez.manuel.j.f@gmail.com',
 'ro.driguezmanuelw@gmail.com',
 'manuelrodrigue.zjf@gmail.com',
 'rodriguez.j.c.manuel@gmail.com',
 'gomezf.hpablo@gmail.com',
 'f.h.pablogomez@gmail.com',
 'rodriguez.manuelj.f@gmail.com',
 'p.hpablo.gomez@gmail.com',
 'pablogome.z.f.h@gmail.com',
 'rodriguezj.cmanuel@gmail.com',
 'pablo.gomez.f.h@gmail.com',
 'jfmanuelrodrigue.z@gmail.com',
 'pablogome.zf.h@gmail.com',
 'j.fmanuelrodriguez@gmail.com',
 'p.hpablogome.z@gmail.com',
 'j.fmanuel.rodriguez@gmail.com',
 'f.h.pablo.gomez@gmail.com',
 'rodrigue.zmanuel.w@gmail.com',
 'p.h.pablo.gomez@gmail.com',
 'rodrigue.z.manuel.w@gmail.com',
 'rodrigue.zmanuelw@gmail.com',
 'rodriguezmanuelj.f@gmail.com',
 'gomezf.h.pablo@gmail.com',
 'f.hpablogomez@gmail.com',
 'j.cmanuelrodriguez@gmail.com',
 'manuel.rodriguez.j.f@gmail.com',
 'p.h.pablogomez@gmail.com

In [ ]:
mails2 = pd.unique(list(rtas_inmob.to.apply(matching.clean_mail))).tolist()
mails2

In [ ]:
print(len(mails), len(mails2))

In [ ]:
rtas_inmob